In [1]:
!pip install ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 821.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.1 MB/s eta 0:00:00


In [16]:
# Download and run the Ollama Linux install script
!curl -fsSL https://ollama.com/install.sh | sh
!command -v systemctl >/dev/null && sudo systemctl stop ollama

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down


In [20]:
import ollama
import os

os.environ.update({'OLLAMA_HOST': '0.0.0.0'})

In [22]:
!echo $OLLAMA_HOST

0.0.0.0


In [21]:
res = ollama.chat(model='llama3', messages=[
    {
        'role': 'user',
        'content': 'why is the sky blue?',
    },
])

ConnectError: [Errno 111] Connection refused

In [23]:
def makeRole(role, action):
    return f"you are {role}, you need to {action}"

def makeContents(content):
    return { 'role': 'user', 'content': content }

In [24]:
def selectOne(prompt):
  return ollama.generate(model='llama3',
                         prompt=f"""
                         There are 2 roles:
                            1) Math Professor in MIT
                            2) History Professor in Havard

                            If you are asked a question, you should  answer the role between  above 1) Math Professor and 2) History Professor.
                            Your answer should only be 'Math Professor' or 'History Professor', no other explanations!

                            <Example>
                            Example 1)
                            [Question] What is the Newton’s law?
                            [Answer] Math Professor

                            Example 2)
                            [Question] How did World War I end?
                            [Answer] History Professor

                            Example 3)
                            [Question] Can you explain sin, cos?
                            [Answer] Math Professor
                            </Example>

                            Question) {prompt}
                            Answer
                          """)

In [25]:
def selectWithAnswer(prompt):
  return ollama.generate(model='llama3',
                         prompt=f"""
                         There are 2 roles:
                            1) Math Professor in MIT
                            2) History Professor in Havard

                          If you asked a question, you job is 2.
                          1) choose the role between above 1) Math Professor and 2) History Professor.
                          2) As a Professor, answer the question step by step

                          ##Response format
                          <role>: <answer>


                          ##Example
                          [Question] what is the Newton's law?
                          [Answer] Math Professor:  Newton's laws of motion are a fundamental part of classical mechanics in physics. Here is a brief overview of each of the three laws:

                          Newton's First Law of Motion
                          Law of Inertia: This law states that an object at rest will stay at rest, and an object in motion will stay in motion with a constant velocity unless acted upon by an external force. This means that an object will not change its motion unless a force is applied.

                          Newton's Second Law of Motion
                          Law of Acceleration: This law states that the acceleration of an object is dependent upon two variables - the net force acting upon the object and the mass of the object. It can be expressed with the equation: [ F = ma ] where ( F ) is the force applied, ( m ) is the mass of the object, and ( a ) is the acceleration produced.

                          Newton's Third Law of Motion
                          Law of Action and Reaction: This law states that for every action, there is an equal and opposite reaction. This means that whenever an object exerts a force on a second object, the second object exerts an equal and opposite force on the first object.

                          [Question] When is the World War 1 end date?
                          [Answer] History Professor: World War I ended on November 11, 1918

                          ##Question
                          [Question] {prompt}??
                          [Answer]
                         """)

In [26]:
import random

def generate_random_numbers(n, m):
    # 1부터 n까지의 자연수 리스트 생성
    numbers = list(range(1, n+1))

    # 리스트에서 m개의 랜덤한 숫자 선택
    random_numbers = random.sample(numbers, m)

    return random_numbers

In [27]:
def sortWithLlama(prerole, n, numlist):

        return ollama.generate(model='llama3',
                           prompt=prerole + '\n\n' + f"""
                              You'll be provided with integer 'n' and a list of integers (length of m).
                              Your job is to sort the list of integers in ascending order and answer the list.
                              !! Just answer the list. Do not answer anything else than the sorted list.

                              <Example>
                              Q) n: 20, list: [4, 2, 4, 5]
                              A) [2, 4, 4, 5]

                              Q) n: 20, list: [19, 18, 19, 1]
                              A) [1, 18, 19, 19]
                              </Example>

                              Q) n: {n}, list: {numlist}
                              A)
                           """)